# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Bañados  ->  E. Bañados  |  ['E. Bañados']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
B. Saha  ->  B. Saha  |  ['B. Saha']
J. Liu  ->  J. Liu  |  ['J. Liu']


G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 78 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2506.06418


extracting tarball to tmp_2506.06418...

 done.


/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2506.06418/00_main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '05_sum' from 'tmp_2506.06418/05_sum.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '04_discuss' from 'tmp_2506.06418/04_discuss.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.22/x64/lib/python3.9/site-packages/arxiv_on_deck

...

Found 94 bibliographic references in tmp_2506.07845/main.bbl.
Issues with the citations
syntax error in line 158: '=' expected
Retrieving document from  https://arxiv.org/e-print/2506.07912


extracting tarball to tmp_2506.07912...

 done.
Retrieving document from  https://arxiv.org/e-print/2506.07970


extracting tarball to tmp_2506.07970...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06418-b31b1b.svg)](https://arxiv.org/abs/2506.06418) | **The Identification of Two JWST/NIRCam-Dark Starburst Galaxies at $z=6.6$ with ALMA**  |
|| F. Sun, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2025-06-10*|
|*Comments*| *14 pages, 5 figures, submitted to an AAS journal. Comments are welcome*|
|**Abstract**|            We analyze two dusty star-forming galaxies at $z=6.6$. These galaxies are selected from the ASPIRE survey, a JWST Cycle-1 medium and ALMA Cycle-9 large program targeting 25 quasars and their environments at $z\simeq6.5 - 6.8$. These galaxies are identified as companions to UV-luminous quasars and robustly detected in ALMA continuum and [C II] emission, yet they are extraordinarily faint at the NIRCam wavelengths (down to $>28.0$ AB mag in the F356W band). They are more obscured than galaxies like Arp220, and thus we refer to them as "NIRCam-dark" starburst galaxies (star formation rate $\simeq 80 - 250\,\mathrm{M}_{\odot}\,\mathrm{yr}^{-1}$). Such galaxies are typically missed by (sub)-millimeter blank-field surveys. From the star-formation history (SFH), we show that the NIRCam-dark galaxies are viable progenitors of massive quiescent galaxies at $z\gtrsim4$ and descendants of UV-luminous galaxies at $z>10$. Although it is hard to constrain their number density from a quasar survey, we conclude that NIRCam-dark galaxies can be as abundant as $n\sim10^{-5.5}$ Mpc$^{-3}$ assuming a light halo occupation model. If true, this would equal to $\sim$30% of the number densities of both the quiescent galaxies at $z\gtrsim4$ and UV-luminous galaxies at $z>10$. We further predict that analogs at $z\sim8$ should exist according to the SFH of early massive quiescent galaxies. However, they may fall below the current detection limits of wide JWST and ALMA surveys, thus remaining "JWST-dark". To fully trace the evolution of massive galaxies and dust-obscured cosmic star formation at $z\gtrsim8$, wide-field JWST/NIRCam imaging and slitless spectroscopic surveys of early protoclusters are essential.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06433-b31b1b.svg)](https://arxiv.org/abs/2506.06433) | **Grand Design Spiral Arms in the Compact, Embedded Protoplanetary Disk of Haro 6-13**  |
|| J. Huang, et al. -- incl., <mark>M. Benisty</mark> |
|*Appeared on*| *2025-06-10*|
|*Comments*| *Accepted by ApJ*|
|**Abstract**|            Millimeter continuum spiral arms have so far only been detected in a handful of protoplanetary disks, and thus we have a limited understanding of the circumstances in which they can form. In particular, substructures in small disks ($R\lessapprox 50$ au) have not been well-characterized in comparison with large disks. We present ALMA 1.3 mm continuum observations of the disk around the T Tauri star Haro 6-13 at a resolution of $\sim0.04''$ ($\sim5$ au). A pair of low-contrast spiral arms are detected at disk radii from $\sim10-35$ au. They can be approximated as Archimedean spirals with pitch angles ranging from $\sim10-30^\circ$. The low value of the disk-averaged spectral index between 1.3 and 3 mm ($\alpha=2.1$) and the high brightness temperatures suggest that the millimeter continuum is likely optically thick and thus may hide sufficient mass for the disk to become gravitationally unstable and form spiral arms. CO observations have shown that Haro 6-13 is surrounded by an envelope, raising the possibility that infall is facilitating spiral arm formation.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.07845-b31b1b.svg)](https://arxiv.org/abs/2506.07845) | **SDSS-V Milky Way Mapper (MWM): ASPCAP Stellar Parameters and Abundances in SDSS-V Data Release 19**  |
|| S. Mészáros, et al. -- incl., <mark>G. Guiglion</mark> |
|*Appeared on*| *2025-06-10*|
|*Comments*| *30 pages, 20 figures, 6 tables, accepted for publication in AJ*|
|**Abstract**|            The goal of this paper is to describe the science verification of Milky Way Mapper (MWM) ASPCAP data products published in Data Release 19 (DR19) of the fifth phase of the Sloan Digital Sky Survey (SDSS-V). We compare MWM ASPCAP atmospheric parameters T$_{\rm eff}$, log g, 24 abundances of 21 elements (carbon, nitrogen, and oxygen have multiple sources for deriving their abundance values) and their uncertainties determined from APOGEE spectrograph spectra with those of the literature and evaluate their accuracy and precision. We also test the zero-point calibration of the v$_{\rm rad}$ derived by the APOGEE Data Reduction Pipeline. This data release contains ASPCAP parameters for 964,989 stars, including all APOGEE-2 targets expanded with new observations of 336,511 stars from the Apache Point Observatory (APO) observed until 4 July 2023. Overall, the new T$_{\rm eff}$ values show excellent agreement with the IRFM scale, while the surface gravities exhibit slight systematic offsets compared to asteroseisimic gravities. The estimated precision of T$_{\rm eff}$ is between 50 and 70 K for giants and 70$-$100 K for dwarfs, while surface gravities are measured with a precision of 0.07$-$0.09 dex for giants. We achieve an estimated precision of 0.02$-$0.04 dex for multiple elements, including metallicity, $\alpha$, Mg, and Si, while the precision of at least 10 elements is better than 0.1 dex.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06453-b31b1b.svg)](https://arxiv.org/abs/2506.06453) | **Mapping the Spatial Distribution of Fast Radio Bursts within their Host Galaxies**  |
|| A. C. Gordon, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-06-10*|
|*Comments*| *50 pages, 10 figures, 7 tables. submitted*|
|**Abstract**|            We present deep optical and near-infrared observations of the host galaxies of 34 fast radio bursts (FRBs) detected by the Commensal Real-time ASKAP Fast Transient (CRAFT) survey on the Australian SKA Pathfinder (ASKAP) to compare the locations of FRBs relative to their host light distributions. Incorporating three additional FRBs from the literature, for a total of four repeating and 33 apparently non-repeating FRBs, we determine their projected galactocentric offsets and find a median of $ 4.2^{+5.7}_{-2.5}$ kpc ($1.0^{+1.5}_{-0.6}r_e$). We model their host surface brightness profiles and develop synthetic spatial distributions of their globular clusters based on host properties. We calculate the likelihood the observed location of each FRB is consistent with the smooth light of its host galaxy, residual (primarily spiral) substructure, or globular cluster distributions. The majority of FRBs favor locations within the disks of their galaxies, while only 7-13\% favor a globular cluster origin, primarily those with galactocentric offsets $\gtrsim3r_e$. At $z<0.15$, where spiral structure is apparent in 86\% of our sample of FRB hosts, we find $\approx 20-46\%$ of FRBs favor an association with spiral arms. Assuming FRBs derive from magnetars, our results support multiple formation channels with the majority of progenitors associated with massive stars and a minority formed through dynamical channels. However, the moderate fraction of FRBs associated with spiral structure indicates that high star formation efficiency of the youngest and most massive stars is not a predominant driver in the production of FRB progenitors.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.06717-b31b1b.svg)](https://arxiv.org/abs/2506.06717) | **The $α$-Attractor E-Model in Warm Inflation: Observational Viability from Planck 2018**  |
|| <mark>B. Saha</mark>, M. K. Nandy |
|*Appeared on*| *2025-06-10*|
|*Comments*| *23 pages with 7 figures*|
|**Abstract**|            We explore the inflationary evolution and observational viability of the $\alpha$-attractor E-model in the framework of warm inflation, focusing on both weak and strong dissipative regimes, with a dissipation coefficient linear in temperature. In the strong regime, we account for the growth of inflaton fluctuations due to coupling with the radiation bath via two different forms for the dissipation enhancement function: one associated with plateau-like potentials, and another motivated by the warm little inflation scenario. Employing slow roll conditions, we analytically derive the expressions for the key inflationary observables, the spectral index $n_s$ and the tensor-to-scalar ratio $r$, in both dissipative regimes. The resulting theoretical trajectories on the $n_s$--$r$ plane are then juxtaposed with the contour plots obtained from Planck 2018 data in order to constrain the model parameter. Our analysis shows that the warm $\alpha$-attractor E-model remains compatible with observations in both dissipative regimes, with dissipation playing a crucial role in shifting the predictions and enlarging the viable parameter space, highlighting observational robustness of the model when extended to warm inflation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.07383-b31b1b.svg)](https://arxiv.org/abs/2506.07383) | **Four ages of rotating stars in the rotation--activity relationship and gyrochronology**  |
|| H. Yang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-06-10*|
|*Comments*| *27 pages, 25 figures, 5 tables, to be published in A&A. The abstract is an abbreviated version, please see the manuscript for details*|
|**Abstract**|            Both of gyrochronology and the rotation-activity relationship are standard techniques to determine the evolution this http URL mismatch in the definition of the evolutionary phases has so far raised many issues in physics and mathematics and hampered the understanding of how the internal dynamo processes affect the observable this http URL address this problem, we seek a unified scheme that shows a one-to-one mapping from gyrochronology to the rotation--activity this http URL obtained the chromospheric activity $R'_{\rm HK}$ of 6846 stars and their rotation periods to investigate the rotation--activity this http URL apply a three-interval model to fit the relationship and find that it is best fitted by three intervals in the range of Ro$<0.7$.We associate those intervals to the convective, gap and interface phases of this http URL,we suggest an additional epoch at late times of the I this http URL further use the three-interval models to fit the period--activity relationship in temperature bins and determine the duration of the transition phase as a function of effective this http URL comparing the critical temperature and period of the g-to-I transition with the slowly rotating sequence of 10 young open clusters whose ages range from 1 Myr to 2.5 Gyr, we conclude that our new model finds the pure I sequence without fast rotating outliers, which defines the zero-age I sequence (ZAIS).We propose that there is an ambiguous consensus on when the I sequence starts to this http URL ambiguity is from the visually convergent sequence of the color--period diagrams in open this http URL visually convergent sequence is younger than the ZAIS and is actually the pre-I sequence that can be associated with the stall of the spin-down. Our results unify the rotation--activity relationship and gyrochonology for the stellar evolution of low-mass stars, for which we coined the ``CgIW" scenario.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.07912-b31b1b.svg)](https://arxiv.org/abs/2506.07912) | **A circularly polarized low-frequency radio burst from the exoplanetary system HD 189733**  |
|| <mark>X. Zhang</mark>, et al. |
|*Appeared on*| *2025-06-10*|
|*Comments*| **|
|**Abstract**|            We aim to detect low-frequency radio emission from exoplanetary systems, which can provide insights into planetary magnetic fields, star-planet interactions, stellar activity, and exo-space weather. The HD 189733 system, hosting a well-studied hot Jupiter, is a prime target for such searches. We conducted NenuFAR imaging observations in the 15-62 MHz range, in order to cover the entire orbital phase of HD 189733 b. Dynamic spectra were generated for the target and other sources in the field, followed by a transient search in the time-frequency plane. The data processing pipeline incorporated direction-dependent calibration and noise characterization to improve sensitivity. We also searched for periodic signals using Lomb-Scargle analysis. A highly circularly polarized radio burst was detected at 50 MHz with a flux density of 1.5 Jy and a significance of 6 sigma at the position of HD 189733. No counterpart was found in Stokes I, likely because the emission is embedded in confusion noise and remains below the detection threshold. The estimated minimum fractional circular polarization of 38% suggests a coherent emission process. A periodicity search revealed no weaker signals linked to the planet's orbital period, the star's rotational period, or the synodic period and harmonic period between them. The burst's properties are consistent with cyclotron maser instability (CMI) emission, but the origin is still ambiguous. The comparison with theoretical models suggests star-planet interaction or stellar activity as potential origins. However, alternative explanations such as contamination from other sources along the line of sight (e.g. the companion M dwarf) or noise fluctuation cannot be ruled out.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2506.07970-b31b1b.svg)](https://arxiv.org/abs/2506.07970) | **The FAST Globular Cluster Pulsar Survey (GC FANS)**  |
|| Y. Lian, et al. -- incl., <mark>Y. Wang</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-06-10*|
|*Comments*| *28 pages, 12 figures, 6 tables. Comments are welcome*|
|**Abstract**|            By January 2025, 60 pulsars were discovered by the Five-hundred-meter Aperture Spherical radio Telescope globular cluster (GC) pulsar survey (GC FANS), with spin periods spanning 1.98 ms to 3960.72 ms. Of these, 55 are millisecond pulsars (MSPs; $P<30$ ms), while 34 are binaries with orbital periods spanning 0.12 days to 466.47 days. This paper describes GC FANS, a deep, thorough search for pulsars in 41 GCs in the FAST sky ($-14^\circ < \delta < 65^\circ$) and describes new discoveries in 14 of them. We present updated timing solutions for M92A, NGC 6712A, M71A, and M71E, all of which are ``spider'' pulsars with short orbital periods. We present new timing solutions for M71B, C, and D. With orbital periods of $\sim$466 and 378 days, M71B and M71C are the widest known GC binaries; these systems resemble the normal wide MSP-He WD systems in the Galactic disk. With a spin period of 101 ms, M71D is in an eccentric ($e\sim$0.63) orbit with an 11-day period and a massive companion; the system has a total mass of $2.63 \pm 0.08 \, M_{\odot}$. These features and its large characteristic age suggest it is a double neutron star system (DNS) formed via massive binary evolution early in the cluster's history, akin to Galactic disk DNSs--unlike other candidate GC DNSs, which typically form dynamically. A comparative analysis of GC pulsar populations within FAST's sky reveals that most clusters (10 of 14) resemble the Galactic disk MSP population, likely due to lower stellar densities.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2506.06418/./figures/hmf_dark.png', 'tmp_2506.06418/./figures/n_z_dark.png', 'tmp_2506.06418/./figures/SFR_Mstar_vs_t.png', 'tmp_2506.06418/./figures/fig1_jw_dark_img.png']
copying  tmp_2506.06418/./figures/hmf_dark.png to _build/html/
copying  tmp_2506.06418/./figures/n_z_dark.png to _build/html/
copying  tmp_2506.06418/./figures/SFR_Mstar_vs_t.png to _build/html/
copying  tmp_2506.06418/./figures/fig1_jw_dark_img.png to _build/html/
exported in  _build/html/2506.06418.md
    + _build/html/tmp_2506.06418/./figures/hmf_dark.png
    + _build/html/tmp_2506.06418/./figures/n_z_dark.png
    + _build/html/tmp_2506.06418/./figures/SFR_Mstar_vs_t.png
    + _build/html/tmp_2506.06418/./figures/fig1_jw_dark_img.png
found figures ['tmp_2506.06433/./continuumoverview.png', 'tmp_2506.06433/./protomidpymodel.png', 'tmp_2506.06433/./radialprofile.png']
copying  tmp_2506.06433/./continuumoverview.png to _build/html/
copying  tmp_2506.06433/./protomidpymodel.png to _build/html/
cop

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\oii}{\mbox{[\ion{O}{2}]}}$
$\newcommand{\oiii}{\mbox{[\ion{O}{3}]}}$
$\newcommand{\hei}{\mbox{\ion{He}{1}}}$
$\newcommand{\heii}{\mbox{\ion{He}{2}}}$
$\newcommand{\ci}{\mbox{[\ion{C}{1}]}}$
$\newcommand{\cii}{\mbox{[\ion{C}{2}]}}$
$\newcommand{\nii}{\mbox{[\ion{N}{2}]}}$
$\newcommand{\sii}{\mbox{[\ion{S}{2}]}}$
$\newcommand{\siii}{\mbox{[\ion{S}{3}]}}$
$\newcommand{\feii}{\mbox{[\ion{Fe}{2}]}}$
$\newcommand{\hb}{\mbox{H\beta}}$
$\newcommand{\ha}{\mbox{H\alpha}}$
$\newcommand{\bra}{\mbox{Br\alpha}}$
$\newcommand{\brb}{\mbox{Br\beta}}$
$\newcommand{\brg}{\mbox{Br\gamma}}$
$\newcommand{\paa}{\mbox{Pa\alpha}}$
$\newcommand{\pab}{\mbox{Pa\beta}}$
$\newcommand{\pag}{\mbox{Pa\gamma}}$
$\newcommand{\pad}{\mbox{Pa\delta}}$
$\newcommand{\lya}{\mbox{Ly\alpha}}$
$\newcommand{\popiii}{\mbox{\ion{Pop}{3}}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\zsp}{\mbox{z_\mathrm{spec}}}$
$\newcommand{\zph}{\mbox{z_\mathrm{phot}}}$
$\newcommand{\lir}{\mbox{L_\mathrm{IR}}}$
$\newcommand{\sap}{\mbox{\texttt{SAPPHIRES}}}$
$\newcommand{\macs}{\mbox{MACS J0416.1--2403}}$
$\newcommand{\textred}[1]{{#1}}$
$\newcommand{\textblue}[1]{{#1}}$
$\newcommand{\thefigure}{\thesection\arabic{figure}}$</div>



<div id="title">

# The Identification of Two JWST/NIRCam-Dark Starburst Galaxies at $z=6.6$ with ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.06418-b31b1b.svg)](https://arxiv.org/abs/2506.06418)<mark>Appeared on: 2025-06-10</mark> -  _14 pages, 5 figures, submitted to an AAS journal. Comments are welcome_

</div>
<div id="authors">

F. Sun, et al.

</div>
<div id="abstract">

**Abstract:** We analyze two dusty star-forming galaxies at $z=6.6$ .These galaxies are selected from the ASPIRE survey, a JWST Cycle-1 medium and ALMA Cycle-9 large program targeting 25 quasars and their environments at $z\simeq6.5 - 6.8$ .These galaxies are identified as companions to UV-luminous quasars and robustly detected in ALMA continuum and $\cii$ emission, yet they are extraordinarily faint at the NIRCam wavelengths (down to $>28.0$ AB mag in the F356W band).They are more obscured than galaxies like Arp220, and thus we refer to them as "NIRCam-dark" starburst galaxies (star formation rate $\simeq 80 - 250$ $\smpy$ ).Such galaxies are typically missed by (sub)-millimeter blank-field surveys.From the star-formation history (SFH), we show that the NIRCam-dark galaxies are viable progenitors of massive quiescent galaxies at $z\gtrsim4$ and descendants of UV-luminous galaxies at $z>10$ .Although it is hard to constrain their number density from a quasar survey, we conclude that NIRCam-dark galaxies can be as abundant as $n\sim10^{-5.5}$ Mpc $^{-3}$ assuming a light halo occupation model.If true, this would equal to $\sim$ 30 \% of the number densities of both the quiescent galaxies at $z\gtrsim4$ and UV-luminous galaxies at $z>10$ .We further predict that analogs at $z\sim8$ should exist according to the SFH of early massive quiescent galaxies.However, they may fall below the current detection limits of wide JWST and ALMA surveys, thus remaining "JWST-dark".To fully trace the evolution of massive galaxies and dust-obscured cosmic star formation at $z\gtrsim8$ , wide-field JWST/NIRCam imaging and slitless spectroscopic surveys of early protoclusters are essential.

</div>

<div id="div_fig1">

<img src="tmp_2506.06418/./figures/hmf_dark.png" alt="Fig4.1" width="50%"/><img src="tmp_2506.06418/./figures/n_z_dark.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** 
The number density constraint of NIRCam-dark galaxies. **Left**: The host halo mass function models of NIRCam-dark starburst galaxies at $z=6.6$.
We consider two models for their host mass distribution, including a light model (dashed orange line) and a heavy model (dashed-dotted purple line; Section \ref{ss:04b_number}).
These two models predict completely different number densities of NIRCam-dark galaxies, while both of them match the existing observations.
For comparison we show the quasar host HMF (red) and overall HMF (black) at the same redshift.
**Right**: Comparison of the number densities with massive quiescent galaxies at $z\sim4$\citep{carnall23b, valentino23, long24a, baker25a}, (U)LIRGs at $z\sim7$\citep[][see Section \ref{ss:04b_number} for details]{barrufet23a, fujimoto24b} and "blue monsters" at $z>10$\citep[][computed from their measured UVLFs in the brightest bins, typically at $M_\mathrm{UV} \sim -21$ mag]{donnan24, finkelstein24, harikane25, whitler25}.
The derived number densities of NIRCam-dark galaxies based on the light and heavy halo occupation model are shown as orange and purple diamonds, respectively.
 (*fig:n*)

</div>
<div id="div_fig2">

<img src="tmp_2506.06418/./figures/SFR_Mstar_vs_t.png" alt="Fig3" width="100%"/>

**Figure 3. -** Stellar mass (left) and SFR (right) versus redshift for NIRCam-dark dusty starburst galaxies (red diamonds) in the context of massive galaxy evolution.
For comparison we show the star-formation histories of massive quiescent galaxies at $z\sim4$ confirmed with JWST spectroscopy (gray circle: $\ci$tealt{glazebrook24} but using the SFH from $\ci$tealt{turner25a}; diamond: $\ci$tealt{degraaff25a} with solar-metallicity SFH model; triangle: $\ci$tealt{carnall23a} but using the SFH from $\ci$tealt{jiz24a}; squares: $\ci$tealt{baker25a}).
We also highlighted two ALMA-confirmed dusty starbursts known at $z>7$ prior to JWST in coral diamonds, including REBELS-25 $\ci$tep[$z=7.3$;][]{hygate23} and MACS0416\_Y1 $\ci$tep[$z=8.3$;][]{tamura19}.
JWST-confirmed luminous galaxies at $z>10$ with low dust content (also known as "blue monsters"; e.g., $\ci$tealt{ziparo23}) are shown in blue circles $\ci$tep[including][]{arrabalharo23,bunker23,carniani24,castellano24,hsiao24a,kokorev25a,naidu25b,zavala25}.
The median evolution track of early massive galaxies in cosmological simulations are shown in colored lines (including FLARES, $\ci$tealt{lovell23}; IllustrisTNG, $\ci$tealt{pillepich18}, $\ci$tealt{nelson19} and $\ci$tealt{hartley23}; THESAN, $\ci$tealt{kannan22} and $\ci$tealt{chittenden25}).
NIRCam-dark dusty starburst galaxies presented by this study are candidates for the progenitors of massive quiescent galaxies at $z\sim4$ and descendants of "blue monsters" at $z>10$.
 (*fig:sfh*)

</div>
<div id="div_fig3">

<img src="tmp_2506.06418/./figures/fig1_jw_dark_img.png" alt="Fig1" width="100%"/>

**Figure 1. -** JWST NIRCam images (F115W, F200W, F356W), ALMA 1.2 mm continuum images and $\ci$i 158 \micron spectra of two NIRCam-dark galaxies at $z_\mathrm{spec}\sim6.6$.
Because the luminous quasar J1526--2050 is visible to the northwest of J1526m2050.C02 (1$\farcs$6 separation), we also show the quasar-subtracted F200W and F356W image of J1526m2020.C02 in the third row, with ALMA continuum contours (purple) at 10 and 20$\sigma$ overlaid.
Image size is 4$\arcsec$$\times$4$\arcsec$.
These galaxies are selected as companions of quasars with ALMA, but remain undetected or faint at JWST/NIRCam wavelengths.
 (*fig:img*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.06418"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Grand Design Spiral Arms in the Compact, Embedded Protoplanetary Disk of Haro 6-13

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.06433-b31b1b.svg)](https://arxiv.org/abs/2506.06433)<mark>Appeared on: 2025-06-10</mark> -  _Accepted by ApJ_

</div>
<div id="authors">

J. Huang, et al. -- incl., <mark>M. Benisty</mark>

</div>
<div id="abstract">

**Abstract:** Millimeter continuum spiral arms have so far only been detected in a handful of protoplanetary disks, and thus we have a limited understanding of the circumstances in which they can form. In particular, substructures in small disks ( $R\lessapprox 50$ au) have not been well-characterized in comparison with large disks. We present ALMA 1.3 mm continuum observations of the disk around the T Tauri star Haro 6-13 at a resolution of $\sim0\farcs04$ ( $\sim5$ au). A pair of low-contrast spiral arms are detected at disk radii from $\sim10-35$ au. They can be approximated as Archimedean spirals with pitch angles ranging from $\sim10-30^\circ$ . The low value of the disk-averaged spectral index between 1.3 and 3 mm ( $\alpha=2.1$ ) and the high brightness temperatures suggest that the millimeter continuum is likely optically thick and thus may hide sufficient mass for the disk to become gravitationally unstable and form spiral arms. CO observations have shown that Haro 6-13 is surrounded by an envelope, raising the possibility that infall is facilitating spiral arm formation.

</div>

<div id="div_fig1">

<img src="tmp_2506.06433/./continuumoverview.png" alt="Fig4" width="100%"/>

**Figure 4. -** Left: 1.3 mm continuum image of Haro 6-13 and the corresponding intensity map as a function of radius and azimuthal angle in disk coordinates. An arcsinh stretch is used for the color scale. The synthesized beam is shown in the lower left corner of the CLEAN image. Right: The CLEAN image with an unsharp mask applied and the corresponding intensity map in $R$, $\theta$ coordinates.  (*fig:continuumoverview*)

</div>
<div id="div_fig2">

<img src="tmp_2506.06433/./protomidpymodel.png" alt="Fig5" width="100%"/>

**Figure 5. -** Top left: 300 random $I(r)$ samples from \texttt{protomidpy} modeling. Center left: CLEAN image of axisymmetric \texttt{protomidpy} model. Bottom left: Corresponding intensity plot in $R$ and $\theta$ coordinates. Top right: Comparison of radial profiles calculated from CLEAN images of the observations and model. Center right: CLEAN image of the continuum residual visibilities created by subtracting the _maximum a posteriori_\texttt{protomidpy} model from the observed visibilities. The purple star marks the disk center. The black dots mark the spiral arm positions identified by \texttt{filfinder}. Bottom right: Plot of the continuum residuals in $R$ and $\theta$ coordinates.   (*fig:protomidpy*)

</div>
<div id="div_fig3">

<img src="tmp_2506.06433/./radialprofile.png" alt="Fig1" width="100%"/>

**Figure 1. -** Azimuthally averaged, deprojected radial profile of the 1.3 mm continuum image. The shaded ribbon shows the scatter in each radial bin divided by the square root of the number of synthesized beams spanning each bin. The inset shows the faint emission at larger radii. (*fig:radialprofile*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.06433"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\unskip\protect\href{https://orcid.org/#1}{\protect\includegraphics[width=8pt,clip]{logo_orcid.png}}}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\ms}{m~s^{-1}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{log~g}$
$\newcommand{\mh}{[M/H]}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\am}{[\alpha/M]}$
$\newcommand{\cm}{[C/M]}$
$\newcommand{\ch}{[C/H]}$
$\newcommand{\cih}{[CI/H]}$
$\newcommand{\nm}{[N/M]}$
$\newcommand{\nh}{[N/H]}$
$\newcommand{\oh}{[O/H]}$
$\newcommand{\nah}{[Na/H]}$
$\newcommand{\mgh}{[Mg/H]}$
$\newcommand{\alh}{[Al/H]}$
$\newcommand{\sih}{[Si/H]}$
$\newcommand{\ph}{[P/H]}$
$\newcommand{\sh}{[S/H]}$
$\newcommand{\kh}{[K/H]}$
$\newcommand{\cah}{[Ca/H]}$
$\newcommand{\tih}{[Ti/H]}$
$\newcommand{\tiih}{[TiII/H]}$
$\newcommand{\vh}{[V/H]}$
$\newcommand{\crh}{[Cr/H]}$
$\newcommand{\mnh}{[Mn/H]}$
$\newcommand{\coh}{[Co/H]}$
$\newcommand{\nih}{[Ni/H]}$
$\newcommand{\cuh}{[Cu/H]}$
$\newcommand{\ceh}{[Ce/H]}$
$\newcommand{\ndh}{[Nd/H]}$
$\newcommand{\xh}{[X/H]}$
$\newcommand{\xm}{[X/M]}$
$\newcommand{\om}{[O/M]}$
$\newcommand{\nam}{[Na/M]}$
$\newcommand{\mgm}{[Mg/M]}$
$\newcommand{\alm}{[Al/M]}$
$\newcommand{\simm}{[Si/M]}$
$\newcommand{\pmm}{[P/M]}$
$\newcommand{\sm}{[S/M]}$
$\newcommand{\km}{[K/M]}$
$\newcommand{\cam}{[Ca/M]}$
$\newcommand{\tim}{[Ti/M]}$
$\newcommand{\vm}{[V/M]}$
$\newcommand{\crm}{[Cr/M]}$
$\newcommand{\mnm}{[Mn/M]}$
$\newcommand{\com}{[Co/M]}$
$\newcommand{\nim}{[Ni/M]}$
$\newcommand{\cum}{[Cu/M]}$
$\newcommand{\cem}{[Ce/M]}$
$\newcommand{\ndm}{[Nd/M]}$
$\newcommand{\snr}{S/N}$
$\newcommand{\vk}{V-K_{\rm s}}$
$\newcommand{\jk}{J-K_{\rm s}}$
$\newcommand{\bprp}{BP-RP}$
$\newcommand{\vmicro}{v_{\rm micro}}$
$\newcommand{\vmacro}{v_{\rm macro}}$
$\newcommand{\vrad}{v_{\rm rad}}$
$\newcommand{\ebv}{E(B-V)}$
$\newcommand{\vsini}{v  \mathrm{sin}  i}$
$\newcommand{\mum}{\mum}$
$\newcommand\hackaltaffiltext{#1#2}$</div>



<div id="title">

# SDSS-V Milky Way Mapper (MWM): ASPCAP Stellar Parameters and Abundances in SDSS-V Data Release 19

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2506.07845-b31b1b.svg)](https://arxiv.org/abs/2506.07845)<mark>Appeared on: 2025-06-10</mark> -  _30 pages, 20 figures, 6 tables, accepted for publication in AJ_

</div>
<div id="authors">

S. Mészáros, et al. -- incl., <mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** The goal of this paper is to describe the science verification of Milky Way Mapper (MWM) ASPCAP data products published in Data Release 19 (DR19) of the fifth phase of the Sloan Digital Sky Survey (SDSS-V). We compare MWM ASPCAP atmospheric parameters $\teff$ , $\logg$ , 24 abundances of 21 elements (carbon, nitrogen, and oxygen have multiple sources for deriving their abundance values) and their uncertainties determined from APOGEE spectrograph spectra with those of the literature and evaluate their accuracy and precision. We also test the zero-point calibration of the $\vrad$ derived by the APOGEE Data Reduction Pipeline. This data release contains ASPCAP parameters for 964,989 stars, including all APOGEE-2 targets expanded with new observations of 336,511 stars from the Apache Point Observatory (APO) observed until 4 July 2023. Overall, the new $\teff$ values show excellent agreement with the IRFM scale, while the surface gravities exhibit slight systematic offsets compared to asteroseisimic gravities. The estimated precision of $\teff$ is between 50 and 70 K for giants and 70 $-$ 100 K for dwarfs, while surface gravities are measured with a precision of 0.07 $-$ 0.09 dex for giants. We achieve an estimated precision of 0.02 $-$ 0.04 dex for multiple elements, including metallicity, $\alpha$ , Mg, and Si, while the precision of at least 10 elements is better than 0.1 dex.

</div>

<div id="div_fig1">

<img src="tmp_2506.07845/./abun_map.png" alt="Fig15" width="100%"/>

**Figure 15. -** Corrected abundances published in DR19 as a function of $\mh$  of stars with $\snr$$>$ 50, no BAD flags, and e$\_$x$\_$h $<$ 0.2. Features seen in the calibrated abundances are discussed in the individual abundance sections. Stripes seen in the low density regions of some abundances (Na, P, K, V, Cr, Co, Cu, Ce, Nd) indicate noding problems at low abundance values. All abundances are plotted, except those that are within 0.03 dex of the grid edge. Abundances of C$_{\rm global}$ and N$_{\rm global}$ are from the global fit. (*abunmap*)

</div>
<div id="div_fig2">

<img src="tmp_2506.07845/./solar_neigh_teff_giants.png" alt="Fig13" width="100%"/>

**Figure 13. -** Raw abundances of giants in the solar neighborhood solar metallicity sample as a function of $\teff$. Stars with $\snr$$>$ 50, no BAD flags, and $-$0.05 $<$\mh$<$ 0.05 are plotted only. The zero-point offset for giants stars were determined from the 4000$-$6000 K range. Abundances of C$_{\rm global}$ and N$_{\rm global}$ are from the global fit. (*solarneigh*)

</div>
<div id="div_fig3">

<img src="tmp_2506.07845/./mwm_apodr17.png" alt="Fig19" width="100%"/>

**Figure 19. -** Comparison between MWM DR19 and APOGEE DR17 abundances as a function of $\teff$  color coded by MWM $\mh$. $\snr$  was restricted to be larger than 50 in both surveys. Abundances of C$_{\rm global}$ and N$_{\rm global}$ are from the global fit. The solid black line shows the running median value. (*mwmapogee*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2506.07845"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

517  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
